# General Notebook Configuration

In [ ]:
# Import General Packages
from tqdm import tqdm
from datasets import Dataset
from typing import Union
from transformers import NllbTokenizerFast, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, EarlyStoppingCallback
import json
import io
import random
import numpy as np
import torch
import os
import datasets
import datetime
import gc

In [ ]:
# Color messages function
def print_mess (mess:str, color="green") -> None:
  match color:
    case "green":
      print (f"\033[92m{mess}\033[0m")
    case "red":
      print (f"\033[91m{mess}\033[0m")
    case "yellow":
      print (f"\033[93m{mess}\033[0m")
    case "white":
      print (f"\033[97m{mess}\033[0m")

In [ ]:
# Environmente identification
try:
  from google.colab import auth
  from googleapiclient.errors import HttpError
  from googleapiclient.discovery import build
  from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload, MediaFileUpload
  IS_COLAB = True
  print_mess ("Running in Google Colaboratoy Environment.")
except ImportError:
  IS_COLAB = False
  print_mess ("Notebook not running in Gooogle Colaboratory Environment.", "yellow")

Running in Google Colaboratoy Environment.


In [ ]:
# Google connection and authentication
if IS_COLAB:
  auth.authenticate_user ()
  drive_service = build ("drive","v3")

In [ ]:
# Set Seeds for Reproducibility
seed = 3407
random.seed (seed)
np.random.seed (seed)
torch.manual_seed (seed)
if torch.cuda.is_available ():
  torch.cuda.manual_seed_all (seed)

# Datasets' Downloading

In [ ]:
# Main forlder ID
main_folder_id = "1jDVh28jOvtTU6Wsz_ZhlYrUT1plT9KXb"

# Common Dataset Folder Name
common_dataset_folder_name = "RMI_Intentions_Dataset_"

# Dataset Names
dataset_names = [
    "RMI_Intentions_Train_Dataset.json",
    "RMI_Intentions_Test_Dataset.json"
]

# Dataset download directory
dataset_download_dir = "/content/Datasets"
os.makedirs(dataset_download_dir, exist_ok=True)

In [ ]:
# Identify most recent Dataset Folder function.
def search_folder (main_folder_id:str, initial_folder_name:str) -> Union[None, str]:
  # Search most recent folder
  match_list = drive_service.files ().list (
      q = (f"'{main_folder_id}' in parents "
           f"and mimeType = 'application/vnd.google-apps.folder' "
           f"and name contains '{initial_folder_name}'"),
      spaces = "drive",
      fields = "nextPageToken, files(id, name, createdTime)",
      orderBy = "createdTime desc"
  ).execute ()

  # Obtain most recent Folder Dataset ID
  try:
    return match_list.get ("files", [])[0]["id"]
  except IndexError:
    return None

In [ ]:
# Identify Dataset function
def search_dataset (dataset_name:str, dataset_folder_id:str) -> Union[None, str]:
  # Search most recent Dataset
  match_list = drive_service.files ().list (
      q = f"'{dataset_folder_id}' in parents and name = '{dataset_name}'",
      spaces = "drive",
      fields = "nextPageToken, files(id, name, createdTime)",
      orderBy = "createdTime desc"
  ).execute ()

  # Obtaint most recent Dataset ID
  try:
    return match_list.get ("files", [])[0]["id"]
  except IndexError:
    return None

In [ ]:
# Download file function
def download_and_save_file (file_id:str, file_name:str, local_path="") -> Union[None, str]:
  # Request to download File
  req = drive_service.files ().get_media (fileId = file_id)
  fh = io.BytesIO ()
  downloader = MediaIoBaseDownload (fh, req)
  done = False

  try:
    # Download Process
    with tqdm (total = 100, unit = "%", desc = f"Downloading {file_name} file") as pbar:
      while done is False:
        status, done = downloader.next_chunk ()
        pbar.update (int (status.progress () * 100) - pbar.n)

    # Writing Process in the Environment
    with open (os.path.join (local_path, file_name), "wb") as f:
      f.write (fh.getvalue())

    fh.seek (0)
    return fh.read().decode("utf-8")
  except HttpError:
    return None

In [ ]:
# Download Dataset Files from Google Drive.
if IS_COLAB:
  try:
    # Identify dataset folder ID
    print_mess ("Search datasets folder...", "yellow")
    dataset_folder_id = search_folder (main_folder_id = main_folder_id,
                                       initial_folder_name = common_dataset_folder_name)
    if dataset_folder_id is None:
      raise ValueError ("The folder was not found.")
    print_mess (f"The most recent dataset folder ID is: {dataset_folder_id}\n")

    # Search Train and Test Datasets
    print_mess ("Search datasets...", "yellow")
    dataset_id_list = []
    for name in dataset_names:
      dataset_id = search_dataset (dataset_folder_id = dataset_folder_id,
                                   dataset_name = name)
      if dataset_id is None:
        raise ValueError (f"The '{name}' was not found.")
      print_mess (f"The file '{name}' was found with ID: {dataset_id}")
      dataset_id_list.append(dataset_id)

    # Download and Save Dataset files
    print_mess ("\nDownload and Save Datasets...", "yellow")
    dataset_file_list = []
    for name in dataset_names:
      dataset_file = download_and_save_file (file_id = dataset_id_list[dataset_names.index(name)],
                                             file_name = name,
                                             local_path = dataset_download_dir)
      if dataset_file is None:
        raise ValueError (f"The '{name}' file could not be downloaded and saved.")
      dataset_file_list.append(dataset_file)
  except Exception as e:
    print_mess ("Something went wrong.", "red")
    print_mess (f"Error: {e}", "yellow")

Search datasets folder...
The most recent dataset folder ID is: 1PdTlqDHuvyHNZ0nkKl_JSr3_BFTr3iF8

Search datasets...
The file 'RMI_Intentions_Train_Dataset.json' was found with ID: 12wBcKG5u0U8LE-wQOllStNyblhRjtvdD
The file 'RMI_Intentions_Test_Dataset.json' was found with ID: 1j-GogIOl0CI2PDA65JKjPlG8AkzwUZFC

Download and Save Datasets...


# Load the Model

In [ ]:
# Detect GPU
if not torch.cuda.is_available ():
  print_mess ("⚠️ No available GPU detected.", "yellow")
else:
  gpu_name = torch.cuda.get_device_name(0)
  major, minor = torch.cuda.get_device_capability(0)
  print_mess(f"🔹 GPU detected: {gpu_name}")
  print_mess(f"🔹 Compute Capability: {major}.{minor}")

  # Validate bfloat16 support
  bf16_support = torch.cuda.is_bf16_supported()
  if bf16_support:
    print_mess("This GPU supports operations in torch.bfloat16 (BF16).", "white")
    print_mess("You can use bnb_4bit_compute_dtype = torch.bfloat16")
  else:
    print_mess("This GPU does not support BF16, use torch.float16 instead.", "white")
    print_mess(" Set bnb_4bit_compute_dtype = torch.float16", "yellow")

🔹 GPU detected: Tesla T4
🔹 Compute Capability: 7.5
This GPU supports operations in torch.bfloat16 (BF16).
You can use bnb_4bit_compute_dtype = torch.bfloat16


In [ ]:
# Max Sequence Length
max_source_length = 300
max_target_length = 1024 # 2048
max_length = max_target_length

In [ ]:
# Define the model name
model_name = 'facebook/nllb-200-distilled-600M'

# Load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained (
    model_name,
    # dtype=torch.bfloat16 if bf16_support else torch.float16,
    # max_length=max_length,
    device_map = "auto"
)
tokenizer = NllbTokenizerFast.from_pretrained (model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

# Processing Datasets

In [ ]:
# Processing Train and Test Datasets Function
def processing_dataset (raw_dataset:dict) -> Union[None, list]:
  processed_data_list = []

  try:
    # Process raw Data
    for conversation in raw_dataset["conversations"]:
      # Input = system_message + user_operator
      system_msg = conversation[0]["value"]
      user_msg = conversation[1]["value"]
      input_text = "Instruction: " + system_msg + " " + user_msg

      # Output = model_answer
      target_text = conversation[2]["value"]

      # Save Process Data
      processed_data_list.append({
          'input_text': input_text,
          'target_text': target_text
      })

    return processed_data_list
  except KeyError:
    return None

In [ ]:
# Dataset Tokenize Function
def tokenize_function (samples:datasets.formatting.formatting.LazyBatch) -> datasets.arrow_dataset.Dataset:

  # Tokenize Input Text
  model_inputs = tokenizer (
      samples["input_text"],
      max_length = max_source_length,
      truncation = True,
      padding = "max_length"
  )

  # Tokenize Target Text (Labels)
  labels = tokenizer (
      samples["target_text"],
      max_length = max_target_length,
      truncation = True,
      padding = "max_length"
  )

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
# Processing Datasets
try:
  print_mess ("Processing Train Dataset...", "yellow")
  train_dataset = Dataset.from_list (processing_dataset (json.loads (dataset_file_list[0])))
  if train_dataset is None:
    raise ValueError ("Train dataset could not be processed.")
  print_mess ("Train Dataset.")
  print (train_dataset)

  print_mess ("\nProcessing Test Dataset...", "yellow")
  test_dataset = Dataset.from_list (processing_dataset (json.loads (dataset_file_list[1])))
  if train_dataset is None:
    raise ValueError ("Test dataset could not be processed.")
  print_mess ("Test Dataset.")
  print (test_dataset)
except Exception as e:
  print_mess ("Something went wrong.", "red")
  print_mess (f"Error: {e}", "yellow")

Processing Train Dataset...
Train Dataset.
Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 242
})

Processing Test Dataset...
Test Dataset.
Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 81
})


In [ ]:
# Tokenize Datasets
print_mess ("Tokenizing Train Dataset...", "yellow")
tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=[
        "input_text",
        "target_text"
    ]
)

print_mess ("\nTokenizing Test Dataset...", "yellow")
tokenized_test_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=[
        "input_text",
        "target_text"
    ]
)

# Present Tokenized Datasets
print_mess ("\nTrain Dataset.")
print (tokenized_train_dataset)
print_mess ("\nTest Dataset.")
print (tokenized_test_dataset)

Tokenizing Train Dataset...


Map:   0%|          | 0/242 [00:00<?, ? examples/s]


Tokenizing Test Dataset...


Map:   0%|          | 0/81 [00:00<?, ? examples/s]


Train Dataset.
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 242
})

Test Dataset.
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 81
})


# Setting the Fine Tuning

In [ ]:
# Define TrainingArguments
training_args = TrainingArguments (
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    # gradient_accumulation_steps = 2,
    warmup_steps = 100, # 5 # 100
    # num_train_epochs = 10,
    max_steps = 1700,
    learning_rate = 0.5e-4,
    logging_steps = 1,
    eval_strategy = "steps",
    eval_steps = 100,
    optim = "adamw_torch", # optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = seed,
    output_dir="outputs",
    report_to = "none",
    # fp16 = not bf16_support,
    # bf16 = bf16_support,
    dataloader_num_workers = 2,
    load_best_model_at_end = True,
    metric_for_best_model = "eval_loss",
)

In [ ]:
# Callback to prevent Overfitting.
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=5,
    early_stopping_threshold=0.0001
)

# Instantiate the Trainer again with the updated training arguments.
trainer = Trainer (
    model = model,
    processing_class = tokenizer,
    train_dataset = tokenized_train_dataset,
    eval_dataset = tokenized_test_dataset,
    args = training_args,
    callbacks=[early_stopping_callback]
)

# Performing fine tuning

In [ ]:
# Current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.234 GB of memory reserved.


In [ ]:
# Start training.
trainer_stats = trainer.train()

Step,Training Loss,Validation Loss
100,4.021800,3.162852
200,1.776700,1.454171
300,0.904400,0.522347
400,0.138200,0.304682
500,0.082100,0.226173
600,0.517200,0.203433
700,0.036200,0.178570
800,0.416200,0.159716
900,0.019800,0.153847
1000,0.034000,0.141139


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4037: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


In [ ]:
# Final memory and Time Stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2945.147 seconds used for training.
49.09 minutes used for training.
Peak reserved memory = 12.604 GB.
Peak reserved memory for training = 7.37 GB.
Peak reserved memory % of max memory = 85.503 %.
Peak reserved memory for training % of max memory = 49.997 %.


# Save Model

---

Ajustar código de abajo, optimizarlo.

In [ ]:
# Save file parameters
def gen_parameters() -> list:
  execution_date = datetime.datetime.now()
  execution_date = execution_date.strftime("%Y%m%d_%H%M%S")
  save_name_folder = f"Translation_Intent_Model_{execution_date}"
  if IS_COLAB:
    save_folder_path = "1GLA-kC8DZZRb4DKEU9l2fZsszLfIGoE7"
  else:
    save_folder_path = "Model_Versions"

  return [save_name_folder, save_folder_path]

In [ ]:
# Create folder function
def create_folder() -> str:
  # Generate folder parameters
  [folder_name, folder_path] = gen_parameters()

  print_mess("Creating folder...", "green")
  if IS_COLAB:
    try:
      folder_metadata = {
          "name": folder_name,
          "mimeType": "application/vnd.google-apps.folder",
          "parents": [folder_path]
      }
      folder = drive_service.files().create(
          body = folder_metadata,
          fields = "id"
      ).execute()
      print_mess(f"Folder '{folder_name}' created correctly with ID: {folder.get('id')}", "green")

      return folder.get("id")
    except HttpError as e:
      print_mess("Something went wrong", "red")
      return ""
  else:
    try:
      folder_path = os.path.join(folder_path, folder_name)
      if not os.path.exists(folder_path):
        os.makedirs(folder_path)
      print_mess(f"Folder '{folder_name}' created correctly", "green")

      return os.path.join(folder_path, folder_name)
    except Exception as e:
      print_mess("Something went wrong", "red")
      return ""

In [ ]:
def save_model_files(model_output_dir, folder_id):
  for filename in os.listdir(model_output_dir):
    filepath = os.path.join(model_output_dir, filename)
    if os.path.isfile(filepath):
      file_metadata = {
          'name': filename,
          'parents': [folder_id]
      }
      media = MediaFileUpload(filepath, mimetype='application/octet-stream')
      file = drive_service.files().create(
              body=file_metadata,
              media_body=media,
              fields='id').execute()
      print_mess(f"File '{filename}' uploaded to Google Drive with ID: {file.get('id')}", "green")

In [ ]:
# Save model and tokenizer and upload to Google Drive
if IS_COLAB:
  print_mess("Saving model and uploading to Google Drive...", "yellow")
  try:
    # Create folder and get its ID
    folder_id = create_folder()
    if not folder_id:
        raise ValueError("Could not create Google Drive folder.")

    # Define output directory in the Colab environment
    output_dir = "model_trained_temp"
    os.makedirs(output_dir, exist_ok=True) # Ensure the directory exists

    # Save model and tokenizer to the temporary local directory
    model.save_pretrained(output_dir, max_shard_size="1GB", safe_serialization=True)
    tokenizer.save_pretrained(output_dir)
    print_mess(f"Model and tokenizer saved successfully in temporary environment directory: {output_dir}", "yellow")

    # Clear CUDA cache
    torch.cuda.empty_cache()
    gc.collect()
    print_mess("Attempted memory cleanup after saving.", "yellow")

    # Upload saved files to Google Drive
    print_mess("Uploading model files to Google Drive...", "white")
    save_model_files(output_dir, folder_id)
    print_mess("Model files saved successfully in Google Drive", "green")

  except Exception as e:
    print_mess("Something went wrong during saving or uploading.", "red")
    print_mess(f"Error: {e}", "yellow")
else:
    print_mess("Not running in Google Colaboratory Environment. Model will be saved locally.", "yellow")
    try:
        # Generate folder parameters for local save
        [folder_name, folder_path_base] = gen_parameters()
        output_dir_local = os.path.join(folder_path_base, folder_name)
        os.makedirs(output_dir_local, exist_ok=True)

        # Save model and tokenizer locally
        model.save_pretrained(output_dir_local)
        tokenizer.save_pretrained(output_dir_local)
        print_mess(f"Model saved successfully in local directory: {output_dir_local}", "yellow")
    except Exception as e:
        print_mess("Something went wrong during local saving.", "red")
        print_mess(f"Error: {e}", "yellow")

Saving model and uploading to Google Drive...
Creating folder...
Folder 'Translation_Intent_Model_20251005_195258' created correctly with ID: 1KgSHoj24Yvesc4hQuUfXoveUDnibTuoM
Model and tokenizer saved successfully in temporary environment directory: model_trained_temp
Attempted memory cleanup after saving.
Uploading model files to Google Drive...
File 'tokenizer.json' uploaded to Google Drive with ID: 17wlwhRpgI39FimBKlmHj8JKha5p46VrL
File 'model-00001-of-00003.safetensors' uploaded to Google Drive with ID: 1VQwI_jCMDnoazmkn8jorLhmQuOqf7Uzj
File 'config.json' uploaded to Google Drive with ID: 1VQu0y0-ES7bMvqLAoLZ8GNlPmyl2efw1
File 'tokenizer_config.json' uploaded to Google Drive with ID: 1DDkyOjpId3y4eSi9kEwYd7wGdqf4d0vf
File 'sentencepiece.bpe.model' uploaded to Google Drive with ID: 1PHEEmHHuIRkYLhZbRZZrE3nQU2uwSAQd
File 'special_tokens_map.json' uploaded to Google Drive with ID: 12DWq0O1by4tlTYG95AL2vW6roW9J3Ueg
File 'generation_config.json' uploaded to Google Drive with ID: 1hJSes

In [ ]:
print_mess("All done!")

All done!
